## Data Preprocessing Steps:
1. Load raw data obtained using curatedTCGAData.R.
2. Select clinical and biospecimen data, combine with RNA expression based on patient ID.
3. Remove columns with NA values > 0.2.
4. Drop rows with "status" and "time" values being NA.
5. Encode categorical variables.
6. Output data for further imputation.

In [163]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
import pandas as pd

# Read the colData file
colData = pd.read_csv('/content/drive/My Drive/3799/tumor_colData.csv',low_memory=False)

# Read the RNASeq2GeneNorm-2 file
RNA = pd.read_csv('/content/drive/My Drive/3799/tumor_BRCA_RNASeq2GeneNorm-20160128.csv')


In [165]:
colData = colData.drop("Unnamed: 0", axis=1)
colData.head(2)

,patientID,years_to_birth,vital_status,days_to_death,days_to_last_followup,tumor_tissue_site,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,...,SigClust.Unsupervised.mRNA,SigClust.Intrinsic.mRNA,miRNA.Clusters,methylation.Clusters,RPPA.Clusters,CN.Clusters,Integrated.Clusters..with.PAM50.,Integrated.Clusters..no.exp.,Integrated.Clusters..unsup.exp.,X60.Gene.classifier.Class.Assignment
0,TCGA-A1-A0SB,70.0,0,NaN,259.0,breast,stage i,t1c,n0,m0,...,NaN,NaN,3.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN
1,TCGA-A1-A0SD,59.0,0,NaN,437.0,breast,stage iia,t2,n0,m0,...,-3.0,-9.0,6.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN


In [166]:
import numpy as np
colData['time'] = np.where(colData['vital_status'] == 0, colData['days_to_last_followup'], colData['days_to_death'])

In [167]:

selected_cols = [  'patientID',
            # Survival data
            'vital_status', 'time',

            # staging parameters
            'pathologic_stage', 'pathology_T_stage', 'pathology_N_stage', 'pathology_M_stage',

            # Demographical variables
            'years_to_birth', 'gender', 'race', 'ethnicity',

            # Lymph_node related
            'number_of_lymph_nodes', 'patient.lymph_node_examined_count',
            'patient.axillary_lymph_node_stage_method_type',
            'patient.number_of_lymphnodes_positive_by_he',

            # ER_Status
            'patient.breast_carcinoma_estrogen_receptor_status',
            'patient.er_level_cell_percentage_category',

            # PR_Status
            'patient.breast_carcinoma_progesterone_receptor_status',
            'patient.progesterone_receptor_level_cell_percent_category',

            # HER_Status
            'patient.lab_proc_her2_neu_immunohistochemistry_receptor_status',
            'patient.her2_erbb_pos_finding_cell_percent_category',

            # Other_status
            'patient.margin_status', 'patient.menopause_status',

            # Variables related to treatment and diagnosis
            'radiation_therapy', 'histological_type',
            'patient.anatomic_neoplasm_subdivisions.anatomic_neoplasm_subdivision',
            'patient.breast_carcinoma_surgical_procedure_name',
            'patient.history_of_neoadjuvant_treatment',
            'patient.initial_pathologic_diagnosis_method',

            # Tumor related
            'patient.biospecimen_cqcf.tumor_samples.tumor_sample.tumor_necrosis_percent',
            'patient.biospecimen_cqcf.tumor_samples.tumor_sample.tumor_nuclei_percent',
            'patient.biospecimen_cqcf.tumor_samples.tumor_sample.tumor_weight'
            ]

selected_data = colData[selected_cols]

In [168]:
# Rename the columns

def rename_columns(col):
    return col.split('.')[-1]

selected_data.columns = selected_data.columns.map(rename_columns)

column_mapping = {
    'vital_status': 'status',
    'years_to_birth':'age'
}

selected_data = selected_data.rename(columns=column_mapping)

In [169]:
selected_data.columns

Index(['patientID', 'status', 'time', 'pathologic_stage', 'pathology_T_stage',
       'pathology_N_stage', 'pathology_M_stage', 'age', 'gender', 'race',
       'ethnicity', 'number_of_lymph_nodes', 'lymph_node_examined_count',
       'axillary_lymph_node_stage_method_type',
       'number_of_lymphnodes_positive_by_he',
       'breast_carcinoma_estrogen_receptor_status',
       'er_level_cell_percentage_category',
       'breast_carcinoma_progesterone_receptor_status',
       'progesterone_receptor_level_cell_percent_category',
       'lab_proc_her2_neu_immunohistochemistry_receptor_status',
       'her2_erbb_pos_finding_cell_percent_category', 'margin_status',
       'menopause_status', 'radiation_therapy', 'histological_type',
       'anatomic_neoplasm_subdivision',
       'breast_carcinoma_surgical_procedure_name',
       'history_of_neoadjuvant_treatment',
       'initial_pathologic_diagnosis_method', 'tumor_necrosis_percent',
       'tumor_nuclei_percent', 'tumor_weight'],
      dt

In [170]:
selected_data.shape

(1093, 32)

In [171]:
# Remove columns with NA values > 0.2
import pandas as pd

# Set threshold for the percentage of NA values
threshold = 0.2

# Calculate the percentage of missing values in each column
na_percentage = selected_data.isnull().mean()

# Get the column names where the percentage of missing values exceeds the threshold
columns_to_drop = na_percentage[na_percentage > threshold].index

# Drop columns with too many missing values
selected_data_filtered = selected_data.drop(columns=columns_to_drop)

In [172]:
selected_data_filtered.shape

(1093, 29)

In [173]:
selected_data_filtered.isnull().sum()

patientID                                                   0
status                                                      0
time                                                        1
pathologic_stage                                            8
pathology_T_stage                                           0
pathology_N_stage                                           0
pathology_M_stage                                           0
age                                                        15
gender                                                      0
race                                                       95
ethnicity                                                 174
number_of_lymph_nodes                                     168
lymph_node_examined_count                                 126
axillary_lymph_node_stage_method_type                     216
number_of_lymphnodes_positive_by_he                       168
breast_carcinoma_estrogen_receptor_status                  48
breast_c

In [174]:
# Save the selected DataFrame to a new CSV file in Google Colab
#selected_data.to_csv('/content/drive/My Drive/3799/selected_col.csv', index=False)

#print("selected DataFrame saved to Google Drive.")

In [175]:
col = RNA.columns
RNA = RNA.T  # Transpose the DataFrame
# Set the values in the first row as column names
RNA.columns = RNA.iloc[0]

# Drop the first row from the DataFrame
RNA = RNA[1:]

RNA = RNA.reset_index(drop=True)
col = col[1:]
RNA.insert(0, 'Hybridization REF', col)
RNA.insert(1, 'patientID', RNA['Hybridization REF'].str[:12])

In [176]:
RNA.head(2)

Unnamed: 0,Hybridization REF,patientID,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AAAU,197.0897,0.0,0.0,102.9634,1.3786,5798.3746,68.2424,8.6165,...,129.5917,1007.7824,1658.4983,258.4941,1208.3738,3507.2482,1894.9342,1180.4565,1.7233,0.0
1,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALI,237.3844,0.0,0.0,70.8646,4.3502,7571.9793,157.6944,0.5438,...,59.8151,448.6134,1343.1213,198.4774,603.5889,5504.6221,1318.6514,406.7428,926.5905,0.0


In [177]:
# Check for NA values in columns
columns_with_na = RNA.columns[RNA.isnull().any()]

if len(columns_with_na) > 0:
    print("Columns with NA values:")
    print(columns_with_na)
else:
    print("No columns with NA values.")

No columns with NA values.


In [178]:
print(colData.shape)
print(RNA.shape)

(1093, 2685)
(1093, 20503)


In [179]:
# only consider RNA (gene expression)
merged_df = pd.merge(selected_data_filtered, RNA, on='patientID')

In [180]:
print(merged_df.shape)

(1093, 20531)


In [181]:
cols = list(merged_df.columns)
cols.insert(0, cols.pop(cols.index('Hybridization REF')))
merged_df = merged_df.loc[:, cols]

In [182]:
merged_df.head(2)

,Hybridization REF,patientID,status,time,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,age,gender,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SB,0,259.0,stage i,t1c,n0,m0,70.0,female,...,95.9568,519.4279,1415.9252,19.3716,1364.5681,6186.7327,1931.2986,1436.1978,552.3144,0.0
1,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SD,0,437.0,stage iia,t2,n0,m0,59.0,female,...,96.27,578.2814,1225.7051,33.0825,868.0837,3559.6725,1278.9678,1195.6,86.0144,0.0


In [183]:
merged_df.shape

(1093, 20531)

In [184]:
merged_df.head(2)

,Hybridization REF,patientID,status,time,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,age,gender,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SB,0,259.0,stage i,t1c,n0,m0,70.0,female,...,95.9568,519.4279,1415.9252,19.3716,1364.5681,6186.7327,1931.2986,1436.1978,552.3144,0.0
1,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SD,0,437.0,stage iia,t2,n0,m0,59.0,female,...,96.27,578.2814,1225.7051,33.0825,868.0837,3559.6725,1278.9678,1195.6,86.0144,0.0


In [185]:
RNA[RNA['Hybridization REF'].str.contains('TCGA-A7-A0CE')]

# but in firebrowse (previous data), 2 samples
# tcga-a7-a0ce  TCGA-A7-A0CE-11A-21R-A089-07  normalized_count
# tcga-a7-a0ce  TCGA-A7-A0CE-01A-11R-A00Z-07  normalized_count

# Why is the problem resolved? As this data filtered only code 01 Primary Solid Tumor
# 11 refers to Solid Tissue Normal

Unnamed: 0,Hybridization REF,patientID,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
125,TCGA-A7-A0CE-01A-11R-A00Z-07,TCGA-A7-A0CE,43.0714,0.339,0.0,138.5134,12.5434,9467.7956,114.925,3.3901,...,80.0068,478.0066,1036.359,49.4957,928.2143,4260.022,1372.6587,1083.8207,16.6116,0.0


In [186]:
duplicate_rows = RNA[RNA.duplicated(subset='patientID')]

if duplicate_rows.empty:
    print("No duplicate values found in the DataFrame.")
else:
    print("Duplicate values found in the DataFrame:")
    print(duplicate_rows)

No duplicate values found in the DataFrame.


In [187]:
# want to use RF method instead

#from sklearn.impute import SimpleImputer

#merged_df_imputed = merged_df

#imputer = SimpleImputer(strategy='mean')
#merged_df_imputed.iloc[:,numerical_features_idx] = imputer.fit_transform(merged_df_imputed.iloc[:,numerical_features_idx])

#categorical_imputer = SimpleImputer(strategy='most_frequent')
#merged_df_imputed.iloc[:,categorical_features_idx] = categorical_imputer.fit_transform(merged_df_imputed.iloc[:,categorical_features_idx])

In [188]:
# drop rows with "status" and "time" value being NA
merged_df_filtered = merged_df.dropna(subset=['status', 'time'])

In [189]:
print(merged_df.shape)
print(merged_df_filtered.shape)

(1093, 20531)
(1092, 20531)


In [190]:
# Find and drop columns with constant values
constant_cols = merged_df_filtered.columns[merged_df_filtered.nunique() <= 1]
new_merged_df_filtered = merged_df_filtered.drop(columns=constant_cols).copy()

In [191]:
print(merged_df_filtered.shape)
print(new_merged_df_filtered.shape)

(1092, 20531)
(1092, 20247)


In [192]:
categorical_features_idx = [4, 5, 6, 7, 9, 10, 11, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
numerical_features_idx = [idx for idx in list(range(0, 20247)) if idx not in categorical_features_idx][4:]
#numerical_features_idx = [8, 12, 13, 15, 27, 28, 29]
#numerical_features_idx += list(range(30, 20247))

In [193]:
new_merged_df_filtered.iloc[:,categorical_features_idx].head()

,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,gender,race,ethnicity,axillary_lymph_node_stage_method_type,breast_carcinoma_estrogen_receptor_status,breast_carcinoma_progesterone_receptor_status,lab_proc_her2_neu_immunohistochemistry_receptor_status,margin_status,menopause_status,radiation_therapy,histological_type,anatomic_neoplasm_subdivision,breast_carcinoma_surgical_procedure_name,history_of_neoadjuvant_treatment,initial_pathologic_diagnosis_method
0,stage i,t1c,n0,m0,female,white,not hispanic or latino,sentinel node biopsy alone,positive,negative,negative,negative,post (prior bilateral ovariectomy or >12 mo si...,NaN,"other, specify",left,lumpectomy,no,fine needle aspiration biopsy
1,stage iia,t2,n0,m0,female,white,not hispanic or latino,sentinel lymph node biopsy plus axillary disse...,positive,positive,negative,positive,NaN,NaN,infiltrating ductal carcinoma,left,lumpectomy,no,core needle biopsy
2,stage i,t1c,n0 (i-),m0,female,white,not hispanic or latino,sentinel lymph node biopsy plus axillary disse...,positive,positive,negative,negative,pre (<6 months since lmp and no prior bilatera...,NaN,mixed histology (please specify),left upper outer quadrant,modified radical mastectomy,no,core needle biopsy
3,stage iia,t2,n0,m0,female,white,not hispanic or latino,sentinel lymph node biopsy plus axillary disse...,positive,positive,negative,negative,pre (<6 months since lmp and no prior bilatera...,NaN,infiltrating ductal carcinoma,left,modified radical mastectomy,no,fine needle aspiration biopsy
4,stage iib,t2,n1a,m0,female,white,not hispanic or latino,sentinel lymph node biopsy plus axillary disse...,positive,positive,negative,negative,post (prior bilateral ovariectomy or >12 mo si...,NaN,"other, specify",right,lumpectomy,no,core needle biopsy


In [194]:
new_merged_df_filtered.iloc[:,numerical_features_idx].head()

,age,number_of_lymph_nodes,lymph_node_examined_count,number_of_lymphnodes_positive_by_he,tumor_necrosis_percent,tumor_nuclei_percent,tumor_weight,A1BG,A1CF,A2BP1,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,70.0,0.0,2.0,0.0,0.0,85.0,500.0,49.1992,0.0,4.505,...,95.9568,519.4279,1415.9252,19.3716,1364.5681,6186.7327,1931.2986,1436.1978,552.3144,0.0
1,59.0,0.0,3.0,0.0,0.0,70.0,500.0,142.2976,0.0,0.3308,...,96.27,578.2814,1225.7051,33.0825,868.0837,3559.6725,1278.9678,1195.6,86.0144,0.0
2,56.0,0.0,8.0,0.0,0.0,90.0,500.0,192.8194,0.0,0.0,...,95.434,726.6146,1018.84,57.5895,960.5923,3007.8157,926.3677,1075.4422,866.1456,0.0
3,54.0,0.0,2.0,0.0,0.0,75.0,500.0,326.0194,0.0,0.4284,...,74.1138,533.3625,1053.4444,94.6772,881.2262,5343.4779,934.3482,508.0867,52.2652,0.8568
4,61.0,1.0,11.0,1.0,0.0,90.0,500.0,168.8309,0.0,4.32,...,48.9605,425.8843,1105.5711,1.44,702.727,2919.6292,1694.8969,569.5257,25.5603,0.0


In [195]:
# One-hot encode categorical variables
categorical_features = new_merged_df_filtered.columns[categorical_features_idx]
new_merged_df_filtered_encoded = pd.get_dummies(new_merged_df_filtered, columns=categorical_features)

In [196]:
new_merged_df_filtered_encoded

,Hybridization REF,patientID,status,time,age,number_of_lymph_nodes,lymph_node_examined_count,number_of_lymphnodes_positive_by_he,tumor_necrosis_percent,tumor_nuclei_percent,...,breast_carcinoma_surgical_procedure_name_simple mastectomy,history_of_neoadjuvant_treatment_no,history_of_neoadjuvant_treatment_yes,initial_pathologic_diagnosis_method_core needle biopsy,initial_pathologic_diagnosis_method_cytology (e.g. peritoneal or pleural fluid),initial_pathologic_diagnosis_method_excisional biopsy,initial_pathologic_diagnosis_method_fine needle aspiration biopsy,initial_pathologic_diagnosis_method_incisional biopsy,"initial_pathologic_diagnosis_method_other method, specify:",initial_pathologic_diagnosis_method_tumor resection
0,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SB,0,259.0,70.0,0.0,2.0,0.0,0.0,85.0,...,0,1,0,0,0,0,1,0,0,0
1,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SD,0,437.0,59.0,0.0,3.0,0.0,0.0,70.0,...,0,1,0,1,0,0,0,0,0,0
2,TCGA-A1-A0SE-01A-11R-A084-07,TCGA-A1-A0SE,0,1321.0,56.0,0.0,8.0,0.0,0.0,90.0,...,0,1,0,1,0,0,0,0,0,0
3,TCGA-A1-A0SF-01A-11R-A144-07,TCGA-A1-A0SF,0,1463.0,54.0,0.0,2.0,0.0,0.0,75.0,...,0,1,0,0,0,0,1,0,0,0
4,TCGA-A1-A0SG-01A-11R-A144-07,TCGA-A1-A0SG,0,434.0,61.0,1.0,11.0,1.0,0.0,90.0,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,TCGA-OL-A66P-01A-11R-A31O-07,TCGA-OL-A66P,0,428.0,75.0,NaN,2.0,NaN,10.0,40.0,...,1,1,0,1,0,0,0,0,0,0
1089,TCGA-B6-A400-01A-11R-A239-07,TCGA-B6-A400,0,215.0,43.0,4.0,5.0,4.0,0.0,70.0,...,0,1,0,0,0,0,0,0,0,1
1090,TCGA-S3-A6ZG-01A-22R-A32P-07,TCGA-S3-A6ZG,0,562.0,71.0,1.0,3.0,1.0,3.0,60.0,...,0,1,0,0,0,1,0,0,0,0
1091,TCGA-E2-A573-01A-11R-A29R-07,TCGA-E2-A573,0,1062.0,48.0,0.0,4.0,0.0,0.0,80.0,...,0,1,0,1,0,0,0,0,0,0


In [197]:
# Save the merged DataFrame to a CSV file
merged_df.to_csv('merged_data.csv', index=False)